In [19]:
import pandas as pd
import numpy as np
import requests
import re
import json
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
import src.clean_functions as cl_f

In [20]:
##Defining cleaning parameters
## params to clean km (max/min)
kmmax=300000
kmmin=2000

## params to clean año column (max/min)
anmax=15
anmin=1

## params to clean precio column (max/min)
prcmax=100000
prcmin=500

## Params low frequency
lf_model=100
lf_brand=200
lf_carro=500

##Bining params
ColumnsBin=['Longitud','Kilometros','Peso','Potencia']
Bins={'Longitud':10,
      'Kilometros':10,
      'Peso':8,
      'Potencia':6
     }

##weighted params
n_km=1
n_ano=1
n_pot=1.2

In [21]:
df1 = pd.DataFrame(pd.read_csv("./cars_clean.csv"))

In [22]:
##Encoding categorical variables
df1['Cambio']=[cl_f.cambio(i) for i in df1['Cambio']]
df1['Traccion']=[cl_f.traccion(i) for i in df1['Traccion']]
df1['Puertas']=[cl_f.puertas(i) for i in df1['Puertas']]
df1['Marchas']=[cl_f.marchas(i) for i in df1['Marchas']]

In [23]:
##Encoding column año
df1['Ano'] = df1['Ano'].str[-4:]
df1=df1.dropna()
df1['Ano'] = df1['Ano'].astype(float)
df1['Ano']=2019-df1['Ano']

In [24]:
##Dropping ouliers rows in km
df1 = df1[df1["Kilometros"]<=kmmax]
df1 = df1[df1["Kilometros"]>=kmmin]

In [25]:
##Dropping ouliers rows for año
df1 = df1[df1["Ano"]<=anmax]
df1 = df1[df1["Ano"]>=anmin]

##Dropping ouliers rows for precio
df1 = df1[df1["Precio"]<=prcmax]
df1 = df1[df1["Precio"]>=prcmin]

##Casting columns
df1['Potencia']=df1['Potencia'].astype('float')
df1['Cilindros']=df1['Cilindros'].astype('float')

In [26]:
##df1=df1.dropna()
df1 = df1[df1["Longitud"] !=0]
df1 = df1[df1["Velocidad"] !=0]
df1 = df1[df1['Cmixto'] !=0]

In [27]:
##removing low frequency Brands and models
df1=cl_f.RemoveLowFreq(df1,"Model",lf_model)
df1=cl_f.RemoveLowFreq(df1,"Brand",lf_brand)
df1=cl_f.RemoveLowFreq(df1,"Carroceria",lf_carro)


In [28]:
##Encoding through get_dummies
df1 = pd.get_dummies(df1, columns=['Combustible'])
df1 = pd.get_dummies(df1, columns=['Brand'])
df1 = pd.get_dummies(df1, columns=['Carroceria'])

In [29]:
ColumnsNorm=['Emisiones','Longitud','Plazas','Peso','Cilindros','Velocidad','Cmixto','Curbano','Extraurbano','0-100km/h','Cilindrada','Cilindros','Parmaximo',"Ano",'Potencia','Kilometros']
##ColumnsNormmaxmin=["Ano",'Potencia','Kilometros']

In [30]:
for col in ColumnsNorm:
    cl_f.normalization(df1,col)

##for col in ColumnsNormmaxmin:
##    cl_f.normalizationmaxmin(df1,col)

In [31]:
###Weighting features
df1["Kilometros"]=df1["Kilometros"]*n_km
df1["Ano"]=df1["Ano"]*n_ano
df1["Potencia"]=df1["Potencia"]*n_pot